# Napari demo

In [3]:
import os 
import warnings
warnings.filterwarnings('ignore')
import napari
import skimage
import matplotlib.pyplot as plt
import numpy as np
import requests
import skimage.feature
import pandas as pd
import trackpy

In [2]:
%gui qt5

## Download the data from Zenodo.

This dataset is a time-lapse of a zebrafish embry acquired by confocal microscopy. See [here](https://zenodo.org/record/1211599#.XZSnAOf7TOR) for more details.

In [3]:
if not os.path.isfile('Sample/cxcr4aMO2_290112.lsm'):
    os.makedirs('Sample/',exist_ok=True)
    myfile = requests.get('https://zenodo.org/record/1211599/files/cxcr4aMO2_290112.lsm?download=1', allow_redirects=True)
    open('Sample/cxcr4aMO2_290112.lsm', 'wb').write(myfile.content)


## Image import and plotting

We rescale the image in z by a factor of 5 to clearly show all features. We only keep the later time points for the demo.

In [4]:
image_time = skimage.io.imread('Sample/cxcr4aMO2_290112.lsm', plugin='tifffile')
image_time = image_time[0,100:150,:,:,:]

In [ ]:
view3 = napari.Viewer(ndisplay=3)
view3.add_image(np.swapaxes(image_time[:,:,:,:],1,2),scale=[1,5,1],colormap ='green',blending='additive');


## Nuclei detection
We use a simple gaussian filtering followed by local maxima detection to find all the nuclei and create an array with time, x, y,z positions.

In [6]:
all_dots = []
for t in range(image_time.shape[0]):
    imG = skimage.filters.gaussian(image_time[t,:,:,:],sigma =2,
                               preserve_range=True)
    locmax = skimage.feature.peak_local_max(imG,min_distance=5,threshold_abs=100)
    newloc = np.stack((t*np.ones(locmax.shape[0]),locmax[:,1],5*locmax[:,0],locmax[:,2]),axis =1)
    all_dots.append(newloc)
spots = np.concatenate(all_dots) 

The array containing spot detections can simply be plotted in Napari. If the dimensions are in the right order, napari naturally plots them correctly.

In [7]:
view3.add_points(spots,face_color='blue',opacity=0.5,size=5)

<Points layer 'Points' at 0x1a1e75b00>

## Tracking

Finally we use [trackpy](https://soft-matter.github.io/trackpy/v0.3.2/) to track the different nuclei. For that we just have to turn our time,x,y,z array into a Pandas dataframe and then call trackpy:

In [8]:
spots_pd = pd.DataFrame(spots,columns=['frame','x','y','z'])
traj = trackpy.link_df(spots_pd,search_range=10)

Frame 49: 172 trajectories present.


We just exclude very short tracks to make the plot more readable:

In [9]:
all_traj = []
for ind in traj.particle.unique():
    curr_traj = traj[traj.particle==ind][['x','y','z']].values
    if curr_traj.shape[0]>5:
        all_traj.append(curr_traj)

And finally we add the trajectories to the plot as lines:

In [10]:
view3.add_shapes(all_traj, shape_type='path',edge_color='red')

<Shapes layer 'Shapes' at 0x1142c4278>